In [1]:
import json
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split

from natsort import natsorted
from torch.utils.data import Dataset
from Get_model_and_data import *


c:\Users\PC\anaconda3\envs\seg\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
array1 = np.random.rand(15)
array2 = np.random.rand(15)

In [20]:
merged_array = np.stack((array1, array2), axis=0)

In [21]:
merged_array

array([[0.10457027, 0.22195662, 0.23159641, 0.54358235, 0.68853768,
        0.82754976, 0.52509291, 0.12439998, 0.75503369, 0.28731748,
        0.50991584, 0.38828309, 0.53668543, 0.9690895 , 0.95571249],
       [0.54950145, 0.30699637, 0.03018004, 0.29364966, 0.23183906,
        0.67555266, 0.26762847, 0.09250946, 0.24644867, 0.35979527,
        0.35570068, 0.46398445, 0.13862736, 0.94550717, 0.31074887]])

In [23]:
class CoefficientModel(nn.Module):
    def __init__(self):
        super(CoefficientModel, self).__init__()
        self.fc1 = nn.Conv2d(2, 1, kernel_size=1, stride=1, padding=0)

    def forward(self, merged_array):
        x = self.fc1(merged_array)
        return x

def global_local_prob_model(train_mask_dir , global_dir, local_dir, device, save_model_path):
    if not os.path.exists(save_model_path):
        os.makedirs(os.path.dirname(save_model_path), exist_ok=True)

    
    array_folder_local = f"{local_dir}/pred_probs_caglar_he_train"
    array_folder_global = f"{global_dir}/pred_probs_caglar_he_train"

    dataset = Global_Local_Dataset(array_folder_local, array_folder_global, train_mask_dir)
    train_loader = DataLoader(dataset, batch_size=2, shuffle=True, drop_last=True)
    

    model = CoefficientModel().to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda epoch: 0.5**(epoch // 15))

    num_epochs = 10
    for epoch in range(num_epochs):
        model.train()
        for merged_array, gt_mask in train_loader:
            merged_array, gt_mask = merged_array.to(device), gt_mask.to(device)
            optimizer.zero_grad()
            output = model(merged_array.float())
            output = output.view(-1, 3456, 3456)
            loss = criterion(output, gt_mask.float())
            loss.backward()
            optimizer.step()
        scheduler.step()
        print(f"Epoch {epoch+1}, Loss: {loss.item()}, Learning Rate: {scheduler.get_last_lr()[0]}")
        for name, param in model.named_parameters():
            if 'weight' in name:
                print(f"Epoch {epoch+1}, Layer: {name}, Coefficients: {param.data}")

    torch.save(model.state_dict(), f"{save_model_path}best_model.pth")
    



def predict_and_save_results(global_dir, local_dir, model_path, device, saved_prob_dir,saved_mask_dir):
    if not os.path.exists(saved_prob_dir):
        os.makedirs(saved_prob_dir, exist_ok=True)
    if not os.path.exists(saved_mask_dir):
        os.makedirs(saved_mask_dir, exist_ok=True)

    model = CoefficientModel().to(device)
    model.load_state_dict(torch.load(model_path))
    model.eval()

    array_folder_local = f"{local_dir}/pred_probs_caglar_he"
    array_folder_global = f"{global_dir}/pred_probs_caglar_he"
    gt_folder = "c:/Users/PC/Desktop/square/label_zoom_1_0_1/train/"  

    dataset = Global_Local_Dataset(array_folder_local, array_folder_global, gt_folder)
    data_loader = DataLoader(dataset, batch_size=1, shuffle=False)

    with torch.no_grad():
        for idx, (merged_array, gt_mask) in enumerate(data_loader):
            merged_array = merged_array.to(device)
            output = model(merged_array.float())
            output_np = output.cpu().numpy().squeeze()

            save_path = os.path.join(saved_prob_dir, f"prediction_{idx}.npy")
            np.save(save_path, output_np)
        

            png_save_path = os.path.join(saved_mask_dir, f"prediction_{idx}.png")
            output_bool = (output_np > 0.5).astype(np.uint8)
            output_img = (output_bool * 255).astype(np.uint8)
            cv2.imwrite(png_save_path, output_img)
        print(f"Prediction masks are  saved")
        print(f"Prediction images are saved")

# Set device and directories
device = "cuda" if torch.cuda.is_available() else "cpu"
train_mask_directory = "c:/Users/PC/Desktop/square/label_zoom_1_0_1/train"
global_dir = "C:/Users/PC/Desktop/Segmentation/out/2024_May_17-17_01_17/"
local_dir = "C:/Users/PC/Desktop/Segmentation/out/2024_May_17-15_40_27"
save_model_path = "Final_model_results/"
saved_prob_dir = f"{save_model_path}prob_arrays/"
saved_mask_dir = f"{save_model_path}mask_arrays/"
# Train the model and save it
global_local_prob_model(train_mask_directory,global_dir, local_dir, device, save_model_path)

# Load the model, make predictions, and save them
predict_and_save_results(global_dir, local_dir, f"{save_model_path}best_model.pth", device, saved_prob_dir,saved_mask_dir)


Epoch 1, Loss: 0.2412302941083908, Learning Rate: 0.001
Epoch 1, Layer: fc1.weight, Coefficients: tensor([[[[-0.0134]],

         [[ 0.4453]]]], device='cuda:0')
Epoch 2, Loss: 0.23053862154483795, Learning Rate: 0.001
Epoch 2, Layer: fc1.weight, Coefficients: tensor([[[[0.0184]],

         [[0.4945]]]], device='cuda:0')
Epoch 3, Loss: 0.12766626477241516, Learning Rate: 0.001
Epoch 3, Layer: fc1.weight, Coefficients: tensor([[[[0.0477]],

         [[0.5393]]]], device='cuda:0')
Epoch 4, Loss: 0.09575638175010681, Learning Rate: 0.001
Epoch 4, Layer: fc1.weight, Coefficients: tensor([[[[0.0752]],

         [[0.5795]]]], device='cuda:0')
Epoch 5, Loss: 0.07092415541410446, Learning Rate: 0.001
Epoch 5, Layer: fc1.weight, Coefficients: tensor([[[[0.1018]],

         [[0.6149]]]], device='cuda:0')
Epoch 6, Loss: 0.048971787095069885, Learning Rate: 0.001
Epoch 6, Layer: fc1.weight, Coefficients: tensor([[[[0.1243]],

         [[0.6452]]]], device='cuda:0')
Epoch 7, Loss: 0.044370979070663

In [17]:
test_Array = np.load("C:/Users/PC/Desktop/Segmentation/IDRiD-Eye-Fundus-Dataset-Lesion-Segmentation/Final_model_results/prob_arrays/prediction_3.npy")

In [22]:
test_Array

array([[-0.12733111, -0.13129711, -0.12131891, ..., -0.07256919,
        -0.07257003, -0.07250434],
       [-0.13381538, -0.11759025, -0.11551949, ..., -0.07522941,
        -0.07523021, -0.0751628 ],
       [-0.13595124, -0.12238556, -0.11541605, ..., -0.07092127,
        -0.07092205, -0.07085466],
       ...,
       [-0.11819002, -0.11602256, -0.11540022, ..., -0.06141147,
        -0.06140807, -0.06169233],
       [-0.11819023, -0.11602274, -0.11540043, ..., -0.06140175,
        -0.06139836, -0.06168264],
       [-0.11817354, -0.11600611, -0.11538377, ..., -0.06220832,
        -0.0622049 , -0.06248736]], dtype=float32)